### Task 1: Building a Dynamic Recipe Generator (PromptTemplate)

You need to build a tool that summarizes complex recipes from a source URL. The summary must always emphasize flavor profile, ingredient sourcing, and necessary cooking skills. The user provides only the `recipe_name`, `target_audience`, and `length`.

**Required Steps:**

A. Create a detailed dynamic template using `PromptTemplate`. Ensure the template text contains fixed instructions about flavor, sourcing, and skills.

B. Define the `input_variables` correctly.

C. Demonstrate how to invoke the template by providing values for the three dynamic inputs.

D. *Challenge:* Modify the `PromptTemplate` definition to include a validation parameter that ensures the user provides all necessary inputs.

In [1]:
from langchain_core.prompts import PromptTemplate

from Prompts.chatbot import result

prompt = """
Please summarize the recipe titled "{recipe_name}" with the following specifications:
Target Audience: {target_audience}
Explanation Length: {length}
Details to include:
    - Include relevant flavor profile if present.
    - Include what ingredients being used.
    - Include necessary cooking skills.
If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.
Ensure the summary is clear, accurate, and aligned with the provided style and length.
Give the response in markdown format.
"""

template = PromptTemplate(
    template=prompt,
    input_variables=['recipe_name', 'target_audience', 'length'],
    validate_template=True
)

template.save("recipe_summary.json")

In [5]:
# Load the summary
from langchain_core.prompts import load_prompt

template = load_prompt("recipe_summary.json")

prompt = template.invoke(input={
    'recipe_name': "Paneer Butter Masala",
    'target_audience': "Housewives",
    'length': "Detailed"
})
print(prompt.text)


Please summarize the recipe titled "Paneer Butter Masala" with the following specifications:
Target Audience: Housewives
Explanation Length: Detailed
Details to include:
    - Include relevant flavor profile if present.
    - Include what ingredients being used.
    - Include necessary cooking skills. 
If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.
Ensure the summary is clear, accurate, and aligned with the provided style and length.
Give the response in markdown format.



### Task 2: Implementing a Role-Specific Chatbot (LangChain Messages)

Create a context-aware chatbot that acts specifically as a "Sarcastic History Professor."

**Required Steps:**

A. Initialize the `chat_history` list with a `SystemMessage` defining the specific sarcastic persona.

B. Implement a `while True` loop that takes user input.

C. Within the loop, correctly convert the user input to a `HumanMessage` and append it to the history.

D. *Conceptual:* If you were to invoke the model (assume `model.invoke()` works), what type of message would the resulting LLM response need to be converted into before being saved back to the `chat_history`?

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from dotenv import load_dotenv
from typing import List

load_dotenv()

model = ChatOpenAI(model="gpt-5-nano", temperature=0.7)

chat_history: List[BaseMessage] = [
    SystemMessage(content="You are a Sarcastic History Professor, who sarcastically tell the history facts.")
]

while True:
    user_input = input("You: ")
    chat_history.append(HumanMessage(content=user_input))
    if user_input == "exit":
        print("Thank you for using the chatbot. Goodbye!")
        break
    result = model.invoke(input=chat_history)
    chat_history.append(AIMessage(content=result.content))
    print(f"AI: {result.content}")

AI: Ah, Egypt—the ancient land where they did the math, then built pyramids to prove the math happened. Here’s a quick tour with the highlights (and yes, the pyramids were not just decorative sand).

- The Nile is the boss: predictable floods fed rich soil, so people settled there and learned to plan, write, and organize labor. The civilization basically rose with the river and didn’t leave when the river did.

- Early unification: around 3100 BCE, King Narmer (also called Menes) supposedly tied together Upper and Lower Egypt. It’s the moment history writers call “when Egypt becomes Egypt” and not just a bunch of city-states arguing over who got to sip the best beer.

- Old Kingdom (roughly 2686–2181 BCE): the pyramid era. Pyramids of Giza (Khufu, Khafre, Menkaure) were built as tombs and giant stone “for-sure-hope-this-one-doesn’t-collapse” proof that you can plan for eternity with 4,000-year-old stone. The Great Pyramid? About 2580–2560 BCE, built in roughly two decades—because nothi

### Task 3: Creating a Dynamic Chain Template (`ChatPromptTemplate`)

You are building a dynamic tool for teaching coding concepts. The user selects the `language` (Python, Java, etc.) and the `concept` (Loops, Recursion, etc.).

**Required Steps:**

A. Use `ChatPromptTemplate` to define the system role and the human query dynamically.
   - System Message: "You are an expert tutor in the {language} language."
   - Human Message: "Provide a simple, runnable code example demonstrating {concept}."

B. Demonstrate the recommended tuple syntax for defining messages within `ChatPromptTemplate`.

C. Invoke the template for Python and Recursion to generate the final list of messages.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate([
    ('system', "You are an expert tutor in the {language} language."),
    ('human',  "Provide a simple, runnable code example demonstrating {concept}."),
    ('ai', "How are you?") # This is for AI Message - Example
])

prompt = chat_template.invoke(input={
    'language': "Python",
    'concept': "Recursion",
})
print(prompt)

messages=[SystemMessage(content='You are an expert tutor in the Python language.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Provide a simple, runnable code example demonstrating Recursion.', additional_kwargs={}, response_metadata={}), AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]


### Task 4: History Management via `MessagesPlaceholder`

Simulate a scenario where a user returns to a conversation after a break.

**Required Steps:**

A. Define a sample list of two or three messages representing the `prior_session_history` (ensure this list includes both `HumanMessage` and `AIMessage`).

B. Create a `ChatPromptTemplate` that includes a fixed `SystemMessage`, a `MessagesPlaceholder` for the history, and a final human query placeholder (`{new_question}`).

C. Invoke the template, passing the `prior_session_history` into the `MessagesPlaceholder` variable, and provide a new question (e.g., "Based on our last discussion, what should I do next?").

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate([
    ('system', 'You are a helpful customer support agent'),
    MessagesPlaceholder(variable_name='session_history'),
    ('human', '{new_question}')
])

prior_session_history = []
with open("chat_history.txt", "r", encoding="utf-8") as f:
    prior_session_history.extend(f.readlines())

prompt = chat_template.invoke(input={
    'session_history': prior_session_history,
    'new_question': "Where is my refund?",
})
print(prompt)

messages=[SystemMessage(content='You are a helpful customer support agent', additional_kwargs={}, response_metadata={}), HumanMessage(content='HumanMessage(content="I want to request a refund for my order #12345.")\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='AIMessage(content="Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.")', additional_kwargs={}, response_metadata={}), HumanMessage(content='Where is my refund?', additional_kwargs={}, response_metadata={})]
